In [ ]:
!pip install tensorflow
!pip install keras
!pip install imblearn
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install tensorflow-addons

In [ ]:
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    Flatten,
    AveragePooling2D,
    Conv2D,
    Dense,
    Dropout,
    ReLU,
    Softmax,
    ELU,
)
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
)
from itertools import cycle

# Optional imports for specific functionalities
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping  # Callback
from imblearn.combine import SMOTETomek

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
WORKING_DIRECTORY = "/content/drive/My Drive/project/input/"#directory where you have the dataset
CLASSES = ['Mild-Demented',
 'Moderate-Demented',
 'Non-Demented',
 'VeryMild-Demented']


In [ ]:
X, y = [], []
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_dataset = datagen.flow_from_directory(WORKING_DIRECTORY, target_size=(176, 208),batch_size=39400, shuffle=True)
X, y = train_dataset.next()

In [ ]:
samples_before = len(X)
print("Images shape :\t", X.shape)
print("Labels shape :\t", y.shape)


In [ ]:
from collections import Counter
print("Number of samples in each class:\t", sorted(Counter(np.argmax(y, axis=1)).items()))
print("Classes Names according to index:\t", train_dataset.class_indices)

In [ ]:
fig = plt.figure(figsize=(10,8))
rows = 4
columns = 4
for i in range(rows * columns):
 fig.add_subplot(rows, columns, i+1)
 num = random.randint(0, len(X)-1 )
 plt.imshow(X[num])
 plt.axis('off')
 plt.title(CLASSES[(np.argmax(y[num]))], fontsize=8)
plt.axis('off')
plt.show()


In [ ]:
X = X.reshape(-1, 176 * 208 * 3)

from imblearn.combine import SMOTEENN  # Imbalanced class handling
smoteenn = SMOTEENN(random_state=42)
X, y  = smoteenn.fit_resample(X, y)
##X, y = SMOTETomek().fit_resample(X, y)
X = X.reshape(-1, 176, 208, 3)
samples_after = len(X)
print(X.shape)
print("Number of samples after Smoteen :\t", sorted(Counter(np.argmax(y, axis=1)).items()))
print("Images shape :\t", X.shape)
print("Labels shape :\t", y.shape)





In [ ]:
fig = plt.figure(figsize=(10,8))
rows = 4
columns = 4
for i in range(rows * columns):
 fig.add_subplot(rows, columns, i+1)
 num = random.randint(samples_before, samples_after - 1 )
 plt.imshow(X[num])
 plt.axis('off')
 plt.title(CLASSES[(np.argmax(y[num]))], fontsize=8)
plt.axis('off')
plt.show()


In [ ]:
X_train, x_val, y_train, y_val = train_test_split(X,y, test_size = 0.2)
X_train, x_test, y_train, y_test = train_test_split(X_train,y_train, test_size = 0.2)
print("Number of samples after splitting into Training, validation & test set\n")
print("Train \t",sorted(Counter(np.argmax(y_train, axis=1)).items()))
print("Validation\t",sorted(Counter(np.argmax(y_val, axis=1)).items()))
print("Test \t",sorted(Counter(np.argmax(y_test, axis=1)).items()))
print("Images shape :\t", X.shape)
print("Labels shape :\t", y.shape)


In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.initializers import glorot_uniform  # Use tensorflow.keras for compatibility

init = glorot_uniform

model = Sequential()

model.add(Input(shape=(176, 208, 3)))

# First convolutional block
model.add(Conv2D(16, 3, kernel_initializer=init, activation='elu'))  # Smaller filter size (3x3) for local features
model.add(AveragePooling2D(pool_size=(2, 2)))

# Second convolutional block - Increased filter count, maintained filter size
model.add(Conv2D(32, 3, kernel_initializer=init, activation='elu'))
model.add(AveragePooling2D(pool_size=(2, 2)))

# Third convolutional block - Introduce max pooling for additional downsampling
model.add(Conv2D(64, 3, kernel_initializer=init, activation='elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # Experiment with MaxPooling for potential improvements

# Fourth convolutional block - Experiment with different filter sizes
model.add(Conv2D(128, 5, kernel_initializer=init, activation='elu'))  # Larger filter size for capturing broader features
model.add(AveragePooling2D(pool_size=(2, 2)))

# Dropout with a slightly higher rate for more aggressive regularization
model.add(Dropout(0.05))

model.add(Flatten())

# Fully connected layer with increased size for more complex decision boundaries
model.add(Dense(512, kernel_initializer=init, activation='elu'))
model.add(Dropout(0.2))  # Higher dropout rate for fully connected layers

model.add(Dense(4, kernel_initializer=init, activation='softmax'))

model.summary()


In [ ]:
import tensorflow_addons as tfa
model.compile(
 optimizer=SGD(learning_rate=0.05),
 loss = tf.keras.losses.CategoricalCrossentropy(name='loss'),
 metrics=[
 tf.keras.metrics.CategoricalAccuracy(name='acc'),
 tf.keras.metrics.AUC(name='auc'),
 tfa.metrics.F1Score(num_classes=4),
 tf.metrics.Precision(name="precision"),
 tf.metrics.Recall(name="recall") ])

In [ ]:
rop_callback = ReduceLROnPlateau(monitor="val_loss", patience=2)
CALLBACKS = [rop_callback]

In [ ]:



valAug = ImageDataGenerator()

batch_size = 8


history = model.fit(
    valAug.flow(X_train, y_train, batch_size=batch_size, shuffle=True),
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=valAug.flow(x_val, y_val, batch_size=batch_size, shuffle=True),
    validation_steps=len(x_test) // batch_size,
    epochs=40,
    callbacks=CALLBACKS
)

In [ ]:
plt.plot(history.history['loss'], 'b')
plt.plot(history.history['val_loss'], 'g')
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["train", "val"])
plt.show()

In [ ]:
plt.plot(history.history['auc'], 'b')
plt.plot(history.history['val_auc'], 'g')
plt.title("Model AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.legend(["train", "val"])
plt.show()




In [ ]:
plt.plot(history.history['precision'], 'b')
plt.plot(history.history['val_precision'], 'g')
plt.title("Model Precision")
plt.xlabel("Epochs")
plt.ylabel("Precision")
plt.legend(["train", "val"])
plt.show()

In [ ]:
plt.plot(history.history['recall'], 'b')
plt.plot(history.history['val_recall'], 'g')
plt.title("Model Recall")
plt.xlabel("Epochs")
plt.ylabel("Recall")
plt.legend(["train", "val"])
plt.show()

In [ ]:
plt.plot(history.history['f1_score'])
plt.plot(history.history['val_f1_score'])
plt.title("Model F1-Score")
plt.xlabel("Epochs")
plt.ylabel("F1-Score")
plt.show()

In [ ]:
pred_labels = model.predict(x_test, batch_size=32)

def roundoff(arr):
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

print(classification_report(y_test, pred_labels, target_names=CLASSES))

In [ ]:
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(y_test, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')

ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels= CLASSES, yticklabels=CLASSES)

plt.title('Confusion Matrix of Model', fontweight='bold', fontsize=14.0)
plt.xlabel('Predictions', fontweight='bold', fontsize=13)
plt.ylabel('Ground Truth', fontweight='bold', fontsize=13)
plt.tight_layout()
plt.show(ax)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], pred_labels[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), pred_labels.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

plt.figure()
lw = 2
plt.plot(
    fpr[2],
    tpr[2],
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.4f)" % roc_auc[2])

plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic ")
plt.legend(loc="lower right")
plt.show()

In [ ]:
n_classes = 4
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(
    fpr["micro"],
    tpr["micro"],
    label="micro-average ROC curve (area = {0:0.4f})".format(roc_auc["micro"]),
    color="deeppink",
    linestyle=":",
    linewidth=4,
)

plt.plot(
    fpr["macro"],
    tpr["macro"],
    label="macro-average ROC curve (area = {0:0.4f})".format(roc_auc["macro"]),
    color="navy",
    linestyle=":",
    linewidth=4,
)

for i in range(n_classes):
    plt.plot(
        fpr[i],
        tpr[i],
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.4f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Replace with your desired directory and filename
model_save_path = '/content/drive/My Drive/project/mypro.h5'

model.save(model_save_path)  # Use absolute path for saving to Google Drive
